<!-- Projeto Desenvolvido na Data Science Academy - www.datascienceacademy.com.br -->
# <font color='blue'>Data Science Academy</font>
## <font color='blue'>Armazenamento e Gestão de Dados com Data Lake e Data Lakehouse</font>
## <font color='blue'>Lab 8</font>
### <font color='blue'>RAG Pipeline Para IA Generativa no Databricks (com Acesso Pelo Google Colab)</font>

## Instalando e Carregando Pacotes

In [1]:
!pip install -q watermark

In [2]:
!pip install -q databricks-cli databricks-sql-connector

In [3]:
!pip install -q databricks-langchain langchain_milvus langchain-huggingface sentence-transformers beautifulsoup4

In [4]:
import os
os.environ["USER_AGENT"] = "LangChain (Google Colab)"

In [5]:
# Imports
import bs4
import sentence_transformers
from langchain_community.document_loaders import WebBaseLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_core.runnables import RunnablePassthrough
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from databricks_langchain import ChatDatabricks
from langchain_milvus import Milvus
import warnings
warnings.filterwarnings('ignore')

## Configurando Credenciais de Acesso ao Databricks

Crie o token conforme mostrado nas aulas.

In [6]:
# Defina as credenciais do Databricks
os.environ["DATABRICKS_HOST"] = "url_Databricks"   # Substitua pela URL do seu Databricks
os.environ["DATABRICKS_TOKEN"] = "token"           # Substitua pelo token gerado

## Extraindo Dados da Web e Criando os Chunks

In [7]:
# Cria o carregador de dados da web
dsa_loader = WebBaseLoader("https://blog.dsacademy.com.br/microsoft-fabric-transformando-dados-em-conhecimento/")

In [8]:
# Executa o carregador e extrai os dados da web
documentos = dsa_loader.load()

In [9]:
# Cria o separador de texto
text_splitter = RecursiveCharacterTextSplitter(chunk_size = 1800, chunk_overlap = 200)

In [10]:
# Aplica o separador e cria os chunks (documentos)
docs = text_splitter.split_documents(documentos)

In [11]:
len(docs)

7

## Carregando o Modelo de Embeddings

Leia sobre modelos de embeddings no Capítulo 17 do curso.

https://huggingface.co/BAAI/bge-small-en-v1.5

In [12]:
# Carrega o modelo de embeddings
embeddings = HuggingFaceEmbeddings(model_name = "BAAI/bge-small-en-v1.5")

## Criando e Carregando o Banco de Dados Vetorial

In [13]:
# Cria o banco de dados vetorial
dsa_vector_db = Milvus.from_documents(documents = docs,
                                      embedding = embeddings,
                                      collection_name = 'dsa_collection',
                                      index_params = {"index_type": "FLAT"},
                                      connection_args = {"uri": "./milvus_dsa.db"})

In [14]:
# Cria o retriever para recuperar os dados do Vector DB
retriever = dsa_vector_db.as_retriever()

In [15]:
retriever

VectorStoreRetriever(tags=['Milvus', 'HuggingFaceEmbeddings'], vectorstore=<langchain_milvus.vectorstores.milvus.Milvus object at 0x7f38f80a5810>, search_kwargs={})

In [16]:
# Testando o Retriever

# Define uma frase
query = "O Que é o Microsoft Fabric?"

# Busca texto similar a frase dentro do banco vetorial
dsa_vector_db.similarity_search(query, k = 1)

[Document(metadata={'description': 'O Microsoft Fabric se destaca como uma plataforma poderosa e versátil, oferecendo um ecossistema completo para manipular, analisar e explorar dados de forma eficiente.', 'language': 'pt-BR', 'pk': 455338279102054401, 'source': 'https://blog.dsacademy.com.br/microsoft-fabric-transformando-dados-em-conhecimento/', 'title': 'Microsoft Fabric - Transformando Dados em Conhecimento - Data Science Academy'}, page_content='A era digital trouxe um crescimento exponencial no volume de dados gerados diariamente. Empresas enfrentam o desafio de transformar esse volume massivo de dados em insights úteis para tomada de decisões estratégicas. Nesse contexto, o Microsoft Fabric se destaca como uma plataforma poderosa e versátil, oferecendo um ecossistema completo para manipular, analisar e explorar dados de forma eficiente.\nO Que é o Microsoft Fabric?\nO Microsoft Fabric é uma plataforma unificada desenvolvida pela Microsoft para integração, análise e exploração de

## Definindo o Endpoint do LLM

In [17]:
# Definimos aqui o endpoint para o LLM no Databricks
llm = ChatDatabricks(endpoint = "databricks-dbrx-instruct", max_tokens = 200)

## Definindo o Prompt Template

In [18]:
# Cria o texto do prompt
PROMPT_TEMPLATE = """
        Humano: Você é um assistente de IA e fornece respostas a perguntas do usuário.

        Use as seguintes informações para fornecer uma resposta concisa à pergunta entre as tags <question>.
        Se você não sabe a resposta, apenas diga que não sabe, não tente inventar uma resposta.

        <context>
        {context}
        </context>

        <question>
        {question}
        </question>

        A resposta deve ser específica e usar apenas informações confiáveis.

        Assistente:"""

In [19]:
# Cria o prompt template
prompt = PromptTemplate(template = PROMPT_TEMPLATE, input_variables = ["context", "question"])

In [20]:
# Cria o retriever
retriever = dsa_vector_db.as_retriever()

In [21]:
# Função para formatar os dados
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

## Definindo o RAG Chain Para o RAG Pipeline

In [22]:
# RAG Chain

rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

O RunnablePassthrough no LangChain é uma classe que simplesmente passa os dados de entrada para a saída sem realizar nenhuma modificação ou processamento. É basicamente um "canal direto" para os dados. Isso é o que faremos com a questão do usuário.

## Executando o Pipeline

In [23]:
# Invoca a chain
resposta = rag_chain.invoke(query)

In [24]:
resposta

'O Microsoft Fabric é uma plataforma unificada desenvolvida pela Microsoft para integração, análise e exploração de dados. Ele combina recursos de engenharia de dados, ciência de dados e business intelligence (BI) em um único ecossistema, facilitando o trabalho de equipes interdisciplinares e reduzindo silos de informações.'

In [25]:
# Nova questão
question = "Como Empresas Podem Utilizar o Microsoft Fabric?"
resposta = rag_chain.invoke(question)
resposta

'Empresas podem utilizar o Microsoft Fabric para diversas aplicações, como automação de processos utilizando pipelines de dados e Machine Learning, análise de dados em tempo real para monitorar KPIs e ajustar estratégias de forma ágil, criação de relatórios personalizados com integração ao Power BI, e escalabilidade da plataforma para crescer junto com a empresa, garantindo performance e custo-benefício.'

In [26]:
# Nova questão
question = "Qual o Papel do Cientista de Dados no Microsoft Fabric?"
resposta = rag_chain.invoke(question)
resposta

'O papel do Cientista de Dados no Microsoft Fabric é explorar e analisar dados, realizar experimentos, treinar modelos de Machine Learning e implantar soluções em um único ecossistema. A plataforma oferece recursos para Cientistas de Dados, como a integração com linguagens populares como Python e R, além de frameworks como TensorFlow e PyTorch. Além disso, a colaboração com Engenheiros de Dados e Analistas de Dados se torna mais eficiente devido à unificação dos módulos no Microsoft Fabric.'

In [27]:
%reload_ext watermark
%watermark -a "Data Science Academy"

Author: Data Science Academy



In [28]:
#%watermark -v -m

In [29]:
#%watermark --iversions

# Fim